# PSF/PRF Photometry on Spitzer Data

The affiliated package *photutils* offers now some functions to perform PSF photometry on astronomical image data. This notebook gives a short introduction on the basis of a Spitzer data set. 

## Introduction

As an improvement, compared to aperture photometry, PSF photometry makes use of instrument related information. To estimate the 
the physical parameters of an astrophysical point source it uses a model how this point souce is imaged by the instrument.
This model is called point spread function (PSF). The PSF can be either obtained from the data itself or modeled by some analytical
function. Additionally there is a second concept, the so called point response function (PRF), which denotes the PSF after discretization with a CCD. We are going to deal mostly with PRFs.

## Preliminaries

First we do our required imports

In [ ]:
import numpy as np

from astropy.table import Table
from astropy.utils.data import get_readable_fileobj
from astropy.io import fits
from astropy.wcs import wcs
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
from photutils import psf

Then we set up the matplotlib environment for this notebook

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## Obtaining and loading the data

The test data are originally from the [Spitzer database](http://irsa.ipac.caltech.edu/data/SPITZER/GLIMPSE/images/I/1.2_mosaics_v2.0/GLON_10-30/GLM_01800+0000_mosaic_I2.fits). However, a copy of this and a source catalog are contained in the [photutils-data](https://github.com/astropy/photutils-datasets) GitHub repository, which we access below.

At first we use `astropy.table` to read the coordinates and fluxes for the sources from the catalog.  Using ``get_readable_fileobj`` with ``cache=True`` allows the data to be downloaded only once at a local copy re-used after:

In [ ]:
catalogurl = 'https://raw.githubusercontent.com/astropy/photutils-datasets/master/data/spitzer_example_catalog.xml'
with get_readable_fileobj(catalogurl, 'binary', cache=True) as f:
    catalog = Table.read(f, format='votable')
fluxes_catalog = catalog['f4_5']

We then read the image data using `astropy.io.fits`. As the image data is given in `MJy/sr`, but in the catalog the units are `mJy`, we have to convert the units. Therefore we use that the image data has a resolution of `(1.2 arcsec)^2 / pixel`. Furthermore we set up a mask to exclude NaN values.

In [ ]:
imgurl = 'https://github.com/astropy/photutils-datasets/raw/master/data/spitzer_example_image.fits'

data = fits.getdata(imgurl, cache=True)
factor = (u.MJy / u.sr * (1.2 * u.arcsec) ** 2 / u.pixel).to(u.mJy / u.pixel)
data *= factor.value
mask = np.isfinite(data)

As the photometry function is currently only able to handle pixel coordinates, we define a WCS transformation to convert the galactic coordinates into pixel coordinates:

In [ ]:
header = fits.getheader(imgurl)
wcs_transform = wcs.WCS(header)
coords = wcs_transform.wcs_world2pix(l, b, 0)

galcoords = SkyCoord(l=catalog['l'], b=catalog['b'], frame='galactic')
pxcoords = np.array(galcoords.to_pixel(wcs_transform))


# It seems that the coordinates are shifted. To have the PRF images properly aligned in the plot below, we correct for this.
pxcoords[0] += 0.18
pxcoords[1] += 0.3

Here is a plot with the source positions overlayed:

In [ ]:
plt.figure(figsize=(16, 12))
plt.imshow(data, cmap='hot', vmin=0, vmax=10, interpolation='None', origin='lower')
plt.xlim(0, 1024)
plt.ylim(0, 512)
plt.plot(pxcoords[0], pxcoords[1], marker="o", markerfacecolor='None', markeredgecolor='y', linestyle='None')
plt.colorbar(orientation='horizontal')
plt.tight_layout()

Now we're ready to estimate the PRF from the data.

### Creating PRFs from image data

To estimate the PRF we work on the complete dataset. We choose a size of 7 pixels and a subsampling of 5 / pixel for the PRF.

In [ ]:
prf_discrete = psf.DiscretePRF.create_from_image(data, pxcoords.T, 7, fluxes=fluxes_catalog, 
                                                 mask=np.logical_not(mask), subsampling=5)

In [ ]:
%timeit -n 3 create_prf(data, pxcoords, 7, fluxes=fluxes_catalog, mask=np.logical_not(mask), subsampling=5)

The object returned by `create_prf` is a `DiscretePRF` object, which is basically a look-up table of the different PRFs at different subpixel positions. But it behaves and can be handled like a usual `astropy.modeling.ParametricModel`. The PRFs are stored in the `DiscretePRF._prf_array` attribute. We make a plot to illustrate this:

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5)
fig.set_size_inches(12, 9)
# Plot kernels
for i in range(5):
    for j in range(5):
        prf_image = prf_discrete._prf_array[i, j]
        im = axes[i, j].imshow(prf_image, interpolation='None')
			
cax = fig.add_axes([0.9, 0.1, 0.03, 0.8])
plt.colorbar(im, cax=cax)
plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05)
plt.show()

As we work on a reduced dataset the PRF images at the boundaries show a few artifacts due to bad statistics. A better handling of outliers could be achieved by setting `mode = 'median'` in the `create_prf` function or reducing the size of the prf.

### Photometry with DiscretePRF

We perform photometry on the first 500 sources of the dataset:

In [ ]:
from photutils.psf import psf_photometry
fluxes_photutils = psf_photometry(data, pxcoords, prf_discrete)

To compare with the catalog results, we make a scatter plot:

In [ ]:
plt.scatter(fluxes_catalog, fluxes_photutils)
plt.loglog()
plt.xlim(40, 500)
plt.ylim(40, 500)
plt.xlabel("Fluxes catalog [mJy]")
plt.ylabel("Fluxes Photutils [mJy]")


### Photometry with Gaussian PSF

We can also perform photometry using an `IntegratedGaussianPRF`:

In [ ]:
psf_gaussian = psf.IntegratedGaussianPRF(1)
fit_photometry = psf.psf_photometry(data, pxcoords, psf_gaussian, fitshape=(8, 8))

In [ ]:
# Measure runtime
%timeit -n 3 psf.psf_photometry(data, pxcoords, psf_gaussian, fitshape=(8, 8))

Again we compare with the catalog:

In [ ]:
plt.scatter(fluxes_catalog, fit_photometry['flux_fit'])
plt.loglog()
plt.xlim(40, 500)
plt.ylim(40, 500)
plt.xlabel("Fluxes catalog [mJy]")
plt.ylabel("Fluxes Gaussian Photutils [mJy]")

### Making residual images

It is also possible to remove PRFs from the data and make residual images:

In [ ]:
residuals = psf.subtract_psf(data.copy(), prf_discrete, pxcoords, fluxes_photutils)

We make a plot of the residual image:

In [ ]:
plt.figure(figsize=(16, 12))
plt.imshow(residuals, cmap='hot', vmin=-1, vmax=10, interpolation='None', origin='lower')
plt.plot(pxcoords.T[0], pxcoords.T[1], marker="o", markerfacecolor='None', markeredgecolor='y', linestyle='None')
plt.xlim(0, 1024)
plt.ylim(0, 512)
plt.colorbar(orientation='horizontal')